In [ ]:
import numpy as np


In [ ]:
datain = np.load('initial_inputs.npy')
dataout = np.load('initial_outputs.npy')

In [ ]:
print(datain)
print(datain.shape)   # Useful if it’s an array
print(type(datain)) 

In [ ]:
print(dataout)
print(dataout.shape)   # Useful if it’s an array
print(type(dataout)) 

# Week 1

In [ ]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from scipy.optimize import minimize
from scipy.stats import norm

# --- Initial data ---
#X_init = np.array([...])  # your 30x4 array
#y_init = np.array([...])  # your 30 outputs
X_init = datain
y_init = dataout
# --- Transform to maximization ---
y_trans = -y_init
y_best = y_trans.max()

In [ ]:
# --- Fit Gaussian Process ---
kernel = Matern(nu=2.5)
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-6, normalize_y=True)
gp.fit(X_init, y_trans)

# --- Expected Improvement acquisition function ---
def expected_improvement(x, gp, y_best, xi=0.01):
    x = np.array(x).reshape(1, -1)
    mu, sigma = gp.predict(x, return_std=True)
    mu = mu[0]
    sigma = sigma[0]
    if sigma == 0.0:
        return 0.0
    imp = mu - y_best - xi
    Z = imp / sigma
    ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
    return -ei  # negative because we minimize in scipy
# --- Expected Improvement acquisition function ---

# --- Optimize acquisition function ---
bounds = [(0,1), (0,1), (0,1), (0,1)]
best_x = None
best_ei = float('inf')

# Multiple random starts to avoid local maxima
for _ in range(20):
    x0 = np.random.rand(4)
    res = minimize(lambda x: expected_improvement(x, gp, y_best),
                   x0=x0, bounds=bounds, method='L-BFGS-B')
    if res.fun < best_ei:
        best_ei = res.fun
        best_x = res.x

x_next = best_x
print("Next hyperparameter set to try:", x_next)


# Week 2

In [ ]:
import numpy as np

X_init = datain
y_init = dataout
# --- Transform to maximization ---
y_trans = -y_init
y_best = y_trans.max()


# Add the new observation
X_new = np.array([[0.999999, 0.999999, 0.910203, 0.738695]])
y_new = np.array([-43.88421604431158])

# Combine with previous data
X_all = np.vstack([X_init, X_new])
y_all = np.concatenate([y_init, y_new])

X_init = X_all
y_init = y_all

print(X_init)
print(y_init)


In [ ]:
# --- Transform to maximization (all values consistent) ---
y_trans_init = -y_init
y_new_trans = -y_new  # transform new point too

# --- Combine old + new data ---
X = np.vstack([X_init, x_new.reshape(1, -1)])
y = np.hstack([y_trans_init, y_new_trans])

# --- Current best ---
y_best = y.max()

print(X)
print(y)
print(y_best)


In [ ]:
# --- Fit Gaussian Process ---
kernel = Matern(nu=2.5)
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-6, normalize_y=True)
gp.fit(X, y)



In [ ]:

# --- Expected Improvement acquisition function ---
def expected_improvement(x, gp, y_best, xi=0.01):
    x = np.array(x).reshape(1, -1)
    mu, sigma = gp.predict(x, return_std=True)
    mu = mu[0]
    sigma = sigma[0]
    if sigma == 0.0:
        return 0.0
    imp = mu - y_best - xi
    Z = imp / sigma
    ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
    return -ei  # negative because we minimize in scipy

In [ ]:

# --- Bounds ---
bounds = [(0, 1)] * 4

# --- Optimize acquisition function ---
best_x = None
best_ei = float('inf')

for _ in range(15):
    # Random start in full space for exploration
    x0 = np.random.rand(4)
    res = minimize(lambda x: expected_improvement(x, gp, y_best),
                   x0=x0, bounds=bounds, method='L-BFGS-B')
    if res.fun < best_ei:
        best_ei = res.fun
        best_x = res.x

for _ in range(5):
    # Random start near previous best for exploitation
    x0 = x_new + np.random.normal(0, 0.05, size=4)
    x0 = np.clip(x0, 0, 1)  # ensure within bounds
    res = minimize(lambda x: expected_improvement(x, gp, y_best),
                   x0=x0, bounds=bounds, method='L-BFGS-B')
    if res.fun < best_ei:
        best_ei = res.fun
        best_x = res.x

x_next = best_x
print("Next hyperparameter set to try:", x_next)


In [ ]:
x_next_6dp = np.round(x_next, 6)
x_next_6dp
print("Next hyperparameter set to try:", x_next_6dp)

# Week 3

In [ ]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from scipy.optimize import minimize
from scipy.stats import norm

# --- Original dataset (30 initial points) ---
X_init = np.array([...])  # 30x4 array of initial hyperparameters
y_init = np.array([...])  # 30 original outputs (negative)

# --- Previous week's best point ---
X_prev_week = np.array([[0.999999, 0.999999, 0.910203, 0.738695]])
y_prev_week = np.array([-43.88421604431158])  # original output

# --- Latest evaluation ---
X_new = np.array([[0.999999, 0.999999, 0.999999, 0.956184]])
y_new = np.array([-53.93723834447932])  # original output

# --- Combine all points ---
X = np.vstack([X_init, X_prev_week, X_new])
y = np.hstack([y_init, y_prev_week, y_new])

# --- Transform to maximization for GP ---
y_trans = -y  # all negative outputs become positive for GP

# --- Current best ---
y_best = y_trans.max()

# --- Fit Gaussian Process ---
kernel = Matern(nu=2.5)
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-6, normalize_y=True)
gp.fit(X, y_trans)

# --- Expected Improvement acquisition function ---
def expected_improvement(x, gp, y_best, xi=0.001):
    x = np.array(x).reshape(1, -1)
    mu, sigma = gp.predict(x, return_std=True)
    mu = mu[0]
    sigma = sigma[0]
    if sigma == 0.0:
        return 0.0
    imp = mu - y_best - xi
    Z = imp / sigma
    ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
    return -ei  # minimize in scipy

# --- Bounds ---
bounds = [(0, 1)] * 4

# --- Optimize acquisition function ---
best_x = None
best_ei = float('inf')

# 1️⃣ Exploitation: many starts near the last best point for fine refinement
for _ in range(10):
    x0 = X_new[0] + np.random.normal(0, 0.02, size=4)  # small noise
    x0 = np.clip(x0, 0, 1)
    res = minimize(lambda x: expected_improvement(x, gp, y_best),
                   x0=x0, bounds=bounds, method='L-BFGS-B')
    if res.fun < best_ei:
        best_ei = res.fun
        best_x = res.x

# 2️⃣ Exploration: a few random starts to cover the rest of the space
for _ in range(5):
    x0 = np.random.rand(4)
    res = minimize(lambda x: expected_improvement(x, gp, y_best),
                   x0=x0, bounds=bounds, method='L-BFGS-B')
    if res.fun < best_ei:
        best_ei = res.fun
        best_x = res.x

x_next = best_x
print("Next hyperparameter set to try:", x_next)


In [ ]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from scipy.optimize import minimize
from scipy.stats import norm

# --- Original dataset (30 initial points) ---

datain = np.load('initial_inputs.npy')
dataout = np.load('initial_outputs.npy')
X_init = datain
y_init = dataout
print(X_init)
print(y_init)

# Summary
All historical points are included:
30 initial points
Previous week’s best (-43.88)
Latest extreme point (-53.93)
Consistent transformation: y_trans = -y
Focus on fine-grained exploitation near the current extreme point (X_new)
Small exploratory starts remain to avoid missing other potential peaks

In [ ]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from scipy.optimize import minimize
from scipy.stats import norm

# --- Original dataset (30 initial points) ---
# X_init = np.array([...])  # (30, 4)
# y_init = np.array([...])  # (30,)

# --- Previous high-performing points ---
X_prev_week = np.array([[0.999999, 0.999999, 0.910203, 0.738695]])  # week 1
y_prev_week = np.array([-43.88421604431158])

X_new = np.array([[0.999999, 0.999999, 0.999999, 0.956184]])        # week 2
y_new = np.array([-53.93723834447932])

# --- Combine all data ---
X = np.vstack([X_init, X_prev_week, X_new])
y = np.hstack([y_init, y_prev_week, y_new])

# --- Transform for maximization (GP expects to maximize) ---
y_trans = -y  # make outputs positive
y_best = y_trans.max()

# --- Fit Gaussian Process ---
kernel = Matern(nu=2.5)
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-6, normalize_y=True)
gp.fit(X, y_trans)

# --- Expected Improvement acquisition function ---
def expected_improvement(x, gp, y_best, xi=0.02):  # <-- slightly higher xi
    x = np.array(x).reshape(1, -1)
    mu, sigma = gp.predict(x, return_std=True)
    mu, sigma = mu[0], sigma[0]
    if sigma == 0.0:
        return 0.0
    imp = mu - y_best - xi
    Z = imp / sigma
    ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
    return -ei  # negative for minimization

# --- Bounds (stay slightly inside [0,1]) ---
epsilon = 0.01
bounds = [(0, 1 - epsilon)] * 4

# --- Optimize the acquisition function ---
best_x = None
best_ei = float('inf')

# 1️⃣ Local search around the last strong point (exploitation)
for _ in range(10):
    x0 = X_new[0] + np.random.normal(0, 0.02, size=4)
    x0 = np.clip(x0, epsilon, 1 - epsilon)
    res = minimize(lambda x: expected_improvement(x, gp, y_best),
                   x0=x0, bounds=bounds, method='L-BFGS-B')
    if res.fun < best_ei:
        best_ei = res.fun
        best_x = res.x

# 2️⃣ Broader search to maintain exploration
for _ in range(5):
    x0 = np.random.rand(4) * (1 - 2*epsilon) + epsilon
    res = minimize(lambda x: expected_improvement(x, gp, y_best),
                   x0=x0, bounds=bounds, method='L-BFGS-B')
    if res.fun < best_ei:
        best_ei = res.fun
        best_x = res.x

x_next = best_x
print("Next hyperparameter set to try:", x_next)


In [ ]:
x_next_6dp = np.round(x_next, 6)
x_next_6dp
print("Next hyperparameter set to try:", x_next_6dp)


# Week 4

<!-- You tested [0.99,0.99,0.99,0.99] → result -53.5903.
Your best so far remains -53.9372 at [0.999999,0.999999,0.999999,0.956184].
So moving slightly inside on all dims produced a slight degradation (worse than the best).
That suggests the true optimum is closer to the previous extreme point, and that the 4th dimension (≈0.956) may be important
it’s not best at 0.99 in that coordinate. The surface seems peaked near the previous best. -->

In [ ]:
# --- Original dataset (30 initial points) ---

datain = np.load('initial_inputs.npy')
dataout = np.load('initial_outputs.npy')
X_init = datain
y_init = dataout

# --- Previous high-performing points ---
X_week1 = np.array([[0.999999, 0.999999, 0.910203, 0.738695]])  # week 1
y_week1 = np.array([-43.88421604431158])

X_week2 = np.array([[0.999999, 0.999999, 0.999999, 0.956184]])        # week 2
y_week2 = np.array([-53.93723834447932])

# --- Combine all data ---
X = np.vstack([X_init, X_week1, X_week2])
y = np.hstack([y_init, y_week1, y_week2])


In [ ]:
X_new = np.array([[0.990000, 0.990000, 0.990000, 0.990000]])        # week 3
y_new = np.array([-53.590344373442626])

# --- Combine all data ---
X = np.vstack([X, X_new])
y = np.hstack([y, y_new])

In [ ]:
print(X)
print(y)

# Tweak to xi only

In [ ]:
# --- Transform for maximization (GP expects to maximize) ---
y_trans = -y  # make outputs positive
y_best = y_trans.max()

# --- Fit Gaussian Process ---
kernel = Matern(nu=2.5)
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-6, normalize_y=True)
gp.fit(X, y_trans)

# --- Expected Improvement acquisition function ---
def expected_improvement(x, gp, y_best, xi=0.002):  # <-- lower
    x = np.array(x).reshape(1, -1)
    mu, sigma = gp.predict(x, return_std=True)
    mu, sigma = mu[0], sigma[0]
    if sigma == 0.0:
        return 0.0
    imp = mu - y_best - xi
    Z = imp / sigma
    ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
    return -ei  # negative for minimization

# --- Bounds (stay slightly inside [0,1]) ---
epsilon = 0.01
bounds = [(0, 1 - epsilon)] * 4

# --- Optimize the acquisition function ---
best_x = None
best_ei = float('inf')

# 1️⃣ Local search around the last strong point (exploitation)
for _ in range(10):
    x0 = X_new[0] + np.random.normal(0, 0.02, size=4)
    x0 = np.clip(x0, epsilon, 1 - epsilon)
    res = minimize(lambda x: expected_improvement(x, gp, y_best),
                   x0=x0, bounds=bounds, method='L-BFGS-B')
    if res.fun < best_ei:
        best_ei = res.fun
        best_x = res.x

# 2️⃣ Broader search to maintain exploration
for _ in range(5):
    x0 = np.random.rand(4) * (1 - 2*epsilon) + epsilon
    res = minimize(lambda x: expected_improvement(x, gp, y_best),
                   x0=x0, bounds=bounds, method='L-BFGS-B')
    if res.fun < best_ei:
        best_ei = res.fun
        best_x = res.x

x_next = best_x
print("Next hyperparameter set to try:", x_next)


Keeps all data so far
✅ Penalizes EI near the edges
✅ Uses safe interior bounds (ε = 1e-3)
✅ Selects a single next point for evaluation

In [ ]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from scipy.optimize import minimize
from scipy.stats import norm

# --- Existing data -------------------------------------------------------------
# X_init: 30x4 initial inputs
# y_init: 30 outputs
# (Paste your original arrays here)
# Example placeholders:
# X_init = np.array([...])
# y_init = np.array([...])

# --- Previous weeks' results ---------------------------------------------------
# X_prev_week = np.array([[0.999999, 0.999999, 0.910203, 0.738695]])
# y_prev_week = np.array([-43.88421604431158])

# X_new = np.array([[0.999999, 0.999999, 0.999999, 0.956184]])
# y_new = np.array([-53.93723834447932])

# X_recent = np.array([[0.99, 0.99, 0.99, 0.99]])
# y_recent = np.array([-53.590344373442626])

# # --- Combine all data ----------------------------------------------------------
# X = np.vstack([X_init, X_prev_week, X_new, X_recent])
# y = np.hstack([y_init, y_prev_week, y_new, y_recent])

# --- Transform for maximization (since y are losses) ---------------------------
y_trans = -y
y_best = y_trans.max()

# --- Fit Gaussian Process -----------------------------------------------------
kernel = Matern(nu=2.5)
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-6, normalize_y=True)
gp.fit(X, y_trans)

# --- Parameters for safe bounds -----------------------------------------------
epsilon = 1e-3
lower, upper = epsilon, 1.0 - epsilon
bounds = [(lower, upper)] * 4

# --- Penalized Expected Improvement function ----------------------------------
def expected_improvement_penalized(x, gp, y_best, xi=0.002, penalty_scale=0.05):
    x = np.array(x).reshape(1, -1)
    mu, sigma = gp.predict(x, return_std=True)
    mu, sigma = mu[0], sigma[0]
    if sigma < 1e-12:
        return 0.0
    imp = mu - y_best - xi
    Z = imp / sigma
    ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
    # --- boundary penalty ---
    closeness = np.minimum(x - lower, upper - x)
    penalty = penalty_scale * np.sum(1.0 / (closeness + 1e-6))
    return -(ei - penalty)  # negative for minimization

# --- Optimization setup -------------------------------------------------------
best_x = None
best_ei = float('inf')

# set X_recent
X_recent = X_new
# 1️⃣ Exploitation: search near latest best region, avoid edges
x_center = X_recent[0]
for _ in range(10):
    x0 = x_center + np.random.normal(0, 0.02, size=4)
    x0 = np.clip(x0, lower, upper)
    res = minimize(lambda x: expected_improvement_penalized(x, gp, y_best, xi=0.005),
                   x0=x0, bounds=bounds, method='L-BFGS-B')
    if res.fun < best_ei:
        best_ei = res.fun
        best_x = res.x

# 2️⃣ Exploration: random restarts to probe other regions
for _ in range(5):
    x0 = np.random.uniform(lower, upper, 4)
    res = minimize(lambda x: expected_improvement_penalized(x, gp, y_best),
                   x0=x0, bounds=bounds, method='L-BFGS-B')
    if res.fun < best_ei:
        best_ei = res.fun
        best_x = res.x

# --- Result -------------------------------------------------------------------
x_next = np.clip(best_x, lower, upper)
print("Next hyperparameter set to try:", x_next)


In [ ]:
# Implement slight tweak
# It keeps the same structure but slightly reduces boundary penalties and biases the optimizer back toward near-edge refinement
# while still allowing global exploration.

In [ ]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from scipy.optimize import minimize
from scipy.stats import norm

# --- Existing data -------------------------------------------------------------
# (Paste your original arrays here)
# X_init = np.array([...])
# y_init = np.array([...])

# --- Transform for maximization -----------------------------------------------
y_trans = -y
y_best = y_trans.max()

# --- Fit Gaussian Process -----------------------------------------------------
kernel = Matern(nu=2.5)
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-6, normalize_y=True)
gp.fit(X, y_trans)

# --- Safe bounds ---------------------------------------------------------------
epsilon = 1e-3
lower, upper = epsilon, 1.0 - epsilon
bounds = [(lower, upper)] * 4

# --- Penalized Expected Improvement -------------------------------------------
def expected_improvement_penalized(x, gp, y_best, xi=0.001, penalty_scale=0.02):
    x = np.array(x).reshape(1, -1)
    mu, sigma = gp.predict(x, return_std=True)
    mu, sigma = mu[0], sigma[0]
    if sigma < 1e-12:
        return 0.0
    imp = mu - y_best - xi
    Z = imp / sigma
    ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
    # boundary penalty: weaker than before
    closeness = np.minimum(x - lower, upper - x)
    penalty = penalty_scale * np.sum(1.0 / (closeness + 1e-6))
    return -(ei - penalty)  # negate for minimization

# --- Optimize acquisition function --------------------------------------------
best_x = None
best_ei = float('inf')

# set X_recent
X_recent = X_new

# 1️⃣ Exploitation: more local restarts near the recent high-performing point
x_center = X_recent[0]
for _ in range(20):  # increased from 10
    x0 = x_center + np.random.normal(0, 0.015, size=4)
    x0 = np.clip(x0, lower, upper)
    res = minimize(lambda x: expected_improvement_penalized(x, gp, y_best,
                                                           xi=0.001,
                                                           penalty_scale=0.02),
                   x0=x0, bounds=bounds, method='L-BFGS-B')
    if res.fun < best_ei:
        best_ei = res.fun
        best_x = res.x

# 2️⃣ Exploration: a few global random restarts
for _ in range(5):
    x0 = np.random.uniform(lower, upper, 4)
    res = minimize(lambda x: expected_improvement_penalized(x, gp, y_best,
                                                           xi=0.001,
                                                           penalty_scale=0.02),
                   x0=x0, bounds=bounds, method='L-BFGS-B')
    if res.fun < best_ei:
        best_ei = res.fun
        best_x = res.x

# --- Result -------------------------------------------------------------------
x_next = np.clip(best_x, lower, upper)
print("Next hyperparameter set to try:", x_next)


In [ ]:
# If it still moves toward midspace, it’s a sign the GP is confident the corner is saturated
# which would be valuable information for your next model iteration.
# This will compute EI without boundary penalty but keep a tiny interior epsilon so you get a proper refinement candidate.

In [ ]:
# assumes gp, X, y_trans already defined as in your pipeline
import numpy as np
from scipy.stats import norm
from scipy.optimize import minimize

epsilon = 1e-6
bounds_edge = [(epsilon, 1-epsilon)]*4

def expected_improvement(x, gp, y_best, xi=0.001):
    x = np.array(x).reshape(1,-1)
    mu, sigma = gp.predict(x, return_std=True)
    sigma = max(sigma[0], 1e-12)
    imp = mu[0] - y_best - xi
    Z = imp / sigma
    return -(imp * norm.cdf(Z) + sigma * norm.pdf(Z))  # negative for minimizer

# center of last-best and small jitter
center = np.array([0.999999, 0.999999, 0.999999, 0.956184])
best_x = None
best_val = 1e9
for _ in range(50):             # many local starts to refine
    x0 = center + np.random.normal(0, 0.005, 4)
    x0 = np.clip(x0, epsilon, 1-epsilon)
    res = minimize(lambda x: expected_improvement(x, gp, y_trans.max(), xi=0.0005),
                   x0=x0, bounds=bounds_edge, method='L-BFGS-B')
    if res.fun < best_val:
        best_val = res.fun
        best_x = res.x

print("Suggested near-edge candidate:", best_x)


# The last few iterations have heavily focused on the boundary — all near [1,1,1,~0.95+].
# The model is now overconfident near that edge and has high uncertainty elsewhere.
# Evaluating the center gives the GP new information in an unexplored region, which helps recalibrate its uncertainty estimates.
# This is healthy exploration, preventing premature convergence on what might be a misleading edge optimum.
# I will stick with [0.49999999 0.5        0.5        0.49999999]

# Week 5

In [ ]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from scipy.optimize import minimize
from scipy.stats import norm
# --- Original dataset (30 initial points) ---

datain = np.load('initial_inputs.npy')
dataout = np.load('initial_outputs.npy')
X_init = datain
y_init = dataout

# --- Previous high-performing points ---
X_week1 = np.array([[0.999999, 0.999999, 0.910203, 0.738695]])  # week 1
y_week1 = np.array([-43.88421604431158])

X_week2 = np.array([[0.999999, 0.999999, 0.999999, 0.956184]])        # week 2
y_week2 = np.array([-53.93723834447932])

X_week3 = np.array([[0.990000, 0.990000, 0.990000, 0.990000]])        # week 3
y_week3 = np.array([-53.590344373442626])

X_week4 = np.array([[0.499999, 0.500000, 0.500000, 0.499999]])        # week 4
y_week4 = np.array([-3.9857608859098153])
# --- Combine all data ---
X = np.vstack([X_init, X_week1, X_week2, X_week3, X_week4])
y = np.hstack([y_init, y_week1, y_week2, y_week3, y_week4])


- The new center point [0.5, 0.5, 0.5, 0.5] produced -3.9858, which is much worse than your best-known results (best so far -53.9372).
- This strongly confirms that the high-performing region is at the upper corner (near the previously tested points ~[1,1,1,~0.95])
- and not in the center of the space. The center is a poor region for the objective.

In [ ]:
# --- Fit Gaussian Process -----------------------------------------------------
kernel = Matern(nu=2.5)
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-6, normalize_y=True)

# assume X, y include the center point; y are original negatives
y_trans = -y
gp.fit(X, y_trans)
y_best = y_trans.max()

# exploit-local refinement around current best
center = np.array([0.999999,0.999999,0.999999,0.956184])
cand_list = np.array([
    center + np.array([0,0,0, delta]) for delta in [0.002, 0.004, -0.004, 0.0005]
])
cand_list = np.clip(cand_list, 0, 1)

# score by EI
from scipy.stats import norm
def ei(x):
    mu, sigma = gp.predict(x.reshape(1,-1), return_std=True)
    if sigma[0] < 1e-12: return 0.0
    imp = mu[0] - y_best - 0.001
    Z = imp / sigma[0]
    return imp * norm.cdf(Z) + sigma[0] * norm.pdf(Z)

scores = [ei(c) for c in cand_list]
best_idx = int(np.argmax(scores))
x_next = cand_list[best_idx]
print("Suggested next (exploit) candidate:", x_next)


# Week 6

- The latest point -54.03 improves slightly over the previous best -53.937.
- The high-performing region is now confirmed at the upper corner, especially the last dimension around 0.956–0.960.
- Incrementing the last coordinate from 0.956184 → 0.960184 gives a small but measurable improvement (-53.94 → -54.03).
- Suggests the optimum may be slightly above 0.960.
- First three dimensions are effectively saturated at 1.0. No gain expected by tweaking them further.
- Exploitation-focused refinement in the 4th dimension is now appropriate.

In [ ]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from scipy.optimize import minimize
from scipy.stats import norm
# --- Original dataset (30 initial points) ---

datain = np.load('initial_inputs.npy')
dataout = np.load('initial_outputs.npy')
X_init = datain
y_init = dataout

# --- Previous high-performing points ---
X_week1 = np.array([[0.999999, 0.999999, 0.910203, 0.738695]])  # week 1
y_week1 = np.array([-43.88421604431158])

X_week2 = np.array([[0.999999, 0.999999, 0.999999, 0.956184]])        # week 2
y_week2 = np.array([-53.93723834447932])

X_week3 = np.array([[0.990000, 0.990000, 0.990000, 0.990000]])        # week 3
y_week3 = np.array([-53.590344373442626])

X_week4 = np.array([[0.499999, 0.500000, 0.500000, 0.499999]])        # week 4
y_week4 = np.array([-3.9857608859098153])

X_week5 = np.array([[0.999999, 0.999999, 0.999999, 0.960184]])        # week 5
y_week5 = np.array([-54.03075969006948])
# --- Combine all data ---
X = np.vstack([X_init, X_week1, X_week2, X_week3, X_week4, X_week5])
y = np.hstack([y_init, y_week1, y_week2, y_week3, y_week4, y_week5])


In [ ]:
print(X)
print("------------")
print(y)

Since the first three dimensions are effectively saturated at 1.0, we only need to refine the 4th dimension in small steps around the current best (0.960184).

In [ ]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from scipy.stats import norm

# --- Assume your existing data is in X, y ---
# Example placeholders:
# X = np.vstack([X_init, previous points...])
# y = np.hstack([y_init, previous outputs...])

# Transform for maximization
y_trans = -y
y_best = y_trans.max()

# Fit Gaussian Process
kernel = Matern(nu=2.5)
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-6, normalize_y=True)
gp.fit(X, y_trans)

# Expected Improvement function
def expected_improvement(x, gp, y_best, xi=0.001):
    x = np.array(x).reshape(1, -1)
    mu, sigma = gp.predict(x, return_std=True)
    sigma = max(sigma[0], 1e-12)
    imp = mu[0] - y_best - xi
    Z = imp / sigma
    ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
    return ei

# --- Generate candidate points along 4th dimension ---
best_corner = np.array([1.0, 1.0, 1.0, 0.960184])
d4_steps = np.array([0.961, 0.962, 0.963, 0.964, 0.965, 0.966])  # next fine steps

candidates = np.array([np.hstack([best_corner[:3], d4]) for d4 in d4_steps])

# --- Score candidates using EI ---
ei_scores = np.array([expected_improvement(x, gp, y_best) for x in candidates])

# --- Rank candidates ---
rank_idx = np.argsort(-ei_scores)  # descending order
top_candidates = candidates[rank_idx]
top_ei = ei_scores[rank_idx]

# Print results
for i, (cand, score) in enumerate(zip(top_candidates, top_ei), 1):
    print(f"Rank {i}: Candidate = {cand}, EI = {score:.6f}")

# Suggested next point
x_next = top_candidates[0]
print("\nSuggested next evaluation point:", x_next)
